In [1]:
#| default_exp py2pyi

# Create delegated pyi

## Setup

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#| export
import ast, sys, inspect, re, os, importlib.util, importlib.machinery

from ast import parse, unparse
from inspect import signature, getsource
from fastcore.utils import *
from fastcore.meta import delegates

## Basics

In [4]:
#| export
def imp_mod(module_path, package=None):
    "Import dynamically the module referenced in `fn`"
    module_path = str(module_path)
    module_name = os.path.splitext(os.path.basename(module_path))[0]
    spec = importlib.machinery.ModuleSpec(module_name, None, origin=module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader = importlib.machinery.SourceFileLoader(module_name, module_path)
    if package is not None: module.__package__ = package
    module.__file__ = os.path.abspath(module_path)
    spec.loader.exec_module(module)
    return module

In [5]:
fn = Path('test_py2pyi.py')

In [47]:
mod = imp_mod(fn)
a = mod.A()
a.h()
a.k() # patch method
a.m() # patch method
a.n() # patch method

1

1

1

1

In [7]:
#| export
def _get_tree(mod):
    return parse(getsource(mod))

In [8]:
tree = _get_tree(mod)

In [9]:
print(unparse(tree)) # now I know what exactly is tree, and tree is from AST ✨✨✨✨

__all__ = ['f', 'g']
from fastcore.meta import delegates
from fastcore.utils import patch

class X(int):
    pass

def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1

@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2

def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1

class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a

class B:
    ...

@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1

@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1

@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1


In [10]:
#| export
@patch
def __repr__(self:ast.AST):
    return unparse(self)

@patch
def _repr_markdown_(self:ast.AST):
    return f"""```python
{self!r}
```"""

In [11]:
for o in enumerate(tree.body): print(o) # now I know what inside tree.body, and maybe each item is a node? ✨✨✨✨

(0, __all__ = ['f', 'g'])
(1, from fastcore.meta import delegates)
(2, from fastcore.utils import patch)
(3, class X(int):
    pass)
(4, def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1)
(5, @delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2)
(6, def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1)
(7, class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a)
(8, class B:
    ...)
(9, @patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1)
(10, @patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1)
(11, @patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1)


In [12]:
node_func = tree.body[4] # yes, Jeremy make each item as a node ✨✨✨✨
node_func 

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
```

In [13]:
#| export
functypes = (ast.FunctionDef,ast.AsyncFunctionDef) # consider both normal func and async func as functypes

In [14]:
isinstance(node_func, functypes)

True

In [15]:
#| export
def _deco_id(d:Union[ast.Name,ast.Attribute])->bool:
    "Get the id for AST node `d`"
    return d.id if isinstance(d, ast.Name) else d.func.id

def has_deco(node:Union[ast.FunctionDef,ast.AsyncFunctionDef], name:str)->bool:
    "Check if a function node `node` has a decorator named `name`"
    return any(_deco_id(d)==name for d in getattr(node, 'decorator_list', []))

In [16]:
nm = 'delegates'
has_deco(node_func, nm)

False

In [17]:
node_del = tree.body[5]
node_del

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

In [18]:
has_deco(node_del, nm)

True

In [49]:
node_pa = tree.body[9]
node_pa
node_pa2 = tree.body[10]
node_pa3 = tree.body[11]
node_pa2
node_pa3

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

```python
@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1
```

```python
@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1
```

In [20]:
has_deco(node_del, 'delegates'), has_deco(node_pa, 'patch')

(True, True)

## Function processing

In [21]:
def _proc_body   (node, mod): print('_proc_body', type(node))
def _proc_func   (node, mod): print('_proc_func', type(node))
def _proc_class  (node, mod): print('_proc_class', type(node))
def _proc_patched(node, mod): print('_proc_patched', type(node))

In [30]:

node_import = tree.body[1]
node_import
node_class = tree.body[3]
node_class
_proc_body(node_import, mod)
_proc_body(node_func, mod)
_proc_func(node_del, mod)
_proc_func(node_pa, mod)
_proc_class(node_class, mod)
_proc_patched(node_pa, mod)

```python
from fastcore.meta import delegates
```

```python
class X(int):
    pass
```

_proc_body <class 'ast.ImportFrom'>
_proc_body <class 'ast.FunctionDef'>
_proc_func <class 'ast.FunctionDef'>
_proc_func <class 'ast.FunctionDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>


In [31]:
#| export
def _get_proc(node):
    if isinstance(node, ast.ClassDef): return _proc_class
    if not isinstance(node, functypes): return None
    if not has_deco(node, 'delegates'): return _proc_body
    if has_deco(node, 'patch'): return _proc_patched
    return _proc_func

In [32]:
#| export
def _proc_tree(tree, mod):
    for node in tree.body:
        proc = _get_proc(node)
        if proc: proc(node, mod) # eventually bring node of AST and mod of runtime together ✨✨✨✨

In [33]:
#| export
def _proc_mod(mod):
    tree = _get_tree(mod)
    _proc_tree(tree, mod)
    return tree

In [34]:
_proc_mod(mod);

_proc_class <class 'ast.ClassDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_func <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>


In [37]:
node_func.name
node_del.name
node_pa.name
node_class.name
node_import.names
node_import.names[0].name

'f'

'g'

'k'

'X'

[delegates]

'delegates'

In [79]:
sym_f = getattr(mod, node_func.name)
node_func, node_func.name, sym_f
sym_g = getattr(mod, node_del.name)
node_del, node_del.name, sym_g
sym_k = getattr(mod, node_pa.name) # 🔥🔥🔥🔥🔥 pylance's getattr can't get patch method from mod the runtime ✨✨✨✨

node_pa, node_pa.name, sym_k
sym_m = getattr(mod, node_pa2.name)
node_pa2, node_pa2.name, sym_m
sym_n = getattr(mod, node_pa3.name) # 
node_pa3, node_pa3.name, sym_n # the output of sym is "ge", this is very strange ✨✨✨✨ ?????


(def f(a: int, b: str='a') -> str:
     """I am f"""
     return 1,
 'f',
 <function test_py2pyi.f(a: int, b: str = 'a') -> str>)

(@delegates(f)
 def g(c, d: X, **kwargs) -> str:
     """I am g"""
     return 2,
 'g',
 <function test_py2pyi.g(c, d: test_py2pyi.X, *, b: str = 'a') -> str>)

(@patch
 @delegates(j)
 def k(self: (A, B), b: bool=False, **kwargs):
     return 1,
 'k',
 None)

(@patch
 @delegates(j)
 def m(self: A, b: bool=False, **kwargs):
     return 1,
 'm',
 None)

(@patch
 def n(self: A, b: bool=False, **kwargs):
     """No delegates here mmm'k?"""
     return 1,
 'n',
 'ge')

In [90]:
# manually change `mod` to `mod.A`: disadvantage is not programmable 🔥🔥🔥🔥🔥
sym_k = getattr(mod.A, node_pa.name) # if mod not working, how about mod.A() ✨✨✨✨ 
node_pa, node_pa.name, sym_k
sym_m = getattr(mod.A, node_pa2.name)
node_pa2, node_pa2.name, sym_m
sym_n = getattr(mod.A, node_pa3.name)
node_pa3, node_pa3.name, sym_n

# to make the above mod.A to be programmable, we need sth like mod.__dict__[node_classA] ✨✨✨✨
node_classA = tree.body[7].name
node_classA
assert mod.A, mod.__dict__[node_classA]

sym_k = getattr(mod.__dict__[node_classA], node_pa.name) # if mod not working, how about mod.A() ✨✨✨✨ 
node_pa, node_pa.name, sym_k
sym_m = getattr(mod.__dict__[node_classA], node_pa2.name)
node_pa2, node_pa2.name, sym_m
sym_n = getattr(mod.__dict__[node_classA], node_pa3.name)
node_pa3, node_pa3.name, sym_n

(@patch
 @delegates(j)
 def k(self: (A, B), b: bool=False, **kwargs):
     return 1,
 'k',
 <function test_py2pyi.A.k(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')>)

(@patch
 @delegates(j)
 def m(self: A, b: bool=False, **kwargs):
     return 1,
 'm',
 <function test_py2pyi.A.m(self: test_py2pyi.A, b: bool = False, *, d: str = 'a')>)

(@patch
 def n(self: A, b: bool=False, **kwargs):
     """No delegates here mmm'k?"""
     return 1,
 'n',
 <function test_py2pyi.A.n(self: test_py2pyi.A, b: bool = False, **kwargs)>)

'A'

(@patch
 @delegates(j)
 def k(self: (A, B), b: bool=False, **kwargs):
     return 1,
 'k',
 <function test_py2pyi.A.k(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')>)

(@patch
 @delegates(j)
 def m(self: A, b: bool=False, **kwargs):
     return 1,
 'm',
 <function test_py2pyi.A.m(self: test_py2pyi.A, b: bool = False, *, d: str = 'a')>)

(@patch
 def n(self: A, b: bool=False, **kwargs):
     """No delegates here mmm'k?"""
     return 1,
 'n',
 <function test_py2pyi.A.n(self: test_py2pyi.A, b: bool = False, **kwargs)>)

In [91]:
sig_f = signature(sym_f)
print(sig_f)

sig_g = signature(sym_g)
print(sig_g)

sig_k = signature(sym_k)
print(sig_k)

sig_m = signature(sym_m)
print(sig_m)

sig_n = signature(sym_n)
print(sig_n)

(a: int, b: str = 'a') -> str
(c, d: test_py2pyi.X, *, b: str = 'a') -> str
(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')
(self: test_py2pyi.A, b: bool = False, *, d: str = 'a')
(self: test_py2pyi.A, b: bool = False, **kwargs)


In [92]:
#| export
def sig2str(sig):
    s = str(sig)
    s = re.sub(r"<class '(.*?)'>", r'\1', s)
    s = re.sub(r"dynamic_module\.", "", s)
    return s

In [93]:
#| export
def ast_args(func):
    sig = signature(func)
    return ast.parse(f"def _{sig2str(sig)}: ...").body[0].args

In [95]:
newargs = ast_args(sym_f)
newargs
newargs = ast_args(sym_g)
newargs
newargs = ast_args(sym_k)
newargs
newargs = ast_args(sym_m)
newargs
newargs = ast_args(sym_n)
newargs

```python
a: int, b: str='a'
```

```python
c, d: test_py2pyi.X, *, b: str='a'
```

```python
self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'
```

```python
self: test_py2pyi.A, b: bool=False, *, d: str='a'
```

```python
self: test_py2pyi.A, b: bool=False, **kwargs
```

In [100]:
node_func.args
node_del.args
node_pa.args
node_pa2.args
node_pa3.args


```python
a: int, b: str='a'
```

```python
c, d: X, **kwargs
```

```python
self: (A, B), b: bool=False, **kwargs
```

```python
self: A, b: bool=False, **kwargs
```

```python
self: A, b: bool=False, **kwargs
```

In [ ]:
node.args = newargs
node

```python
@delegates(f)
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    return 2
```

In [ ]:
#| export
def _body_ellip(n: ast.AST):
    stidx = 1 if isinstance(n.body[0], ast.Expr) and isinstance(n.body[0].value, ast.Str) else 0
    n.body[stidx:] = [ast.Expr(ast.Constant(...))]

In [ ]:
_body_ellip(node)
node

```python
@delegates(f)
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

In [ ]:
#| export
def _update_func(node, sym):
    """Replace the parameter list of the source code of a function `f` with a different signature.
    Replace the body of the function with just `pass`, and remove any decorators named 'delegates'"""
    node.args = ast_args(sym)
    _body_ellip(node)
    node.decorator_list = [d for d in node.decorator_list if _deco_id(d) != 'delegates']

In [ ]:
tree = _get_tree(mod)
node = tree.body[5]
node

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

In [ ]:
_update_func(node, sym)
node

```python
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

In [ ]:
#| export
def _proc_body(node, mod): _body_ellip(node)

In [ ]:
#| export
def _proc_func(node, mod):
    sym = getattr(mod, node.name)
    _update_func(node, sym)

In [ ]:
tree = _proc_mod(mod)
tree.body[5]

_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>


```python
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

## Patch

In [ ]:
node = tree.body[9]
node

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

In [ ]:
ann = node.args.args[0].annotation

In [ ]:
if hasattr(ann, 'elts'): ann = ann.elts[0]

In [ ]:
nm = ann.id
nm

'A'

In [ ]:
cls = getattr(mod, nm)
sym = getattr(cls, node.name)

In [ ]:
sig2str(signature(sym))

"(self: (test_py2pyi.A, test_py2pyi.B), b: bool = False, *, d: str = 'a')"

In [ ]:
_update_func(node, sym)

In [ ]:
node

```python
@patch
def k(self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'):
    ...
```

In [ ]:
#| export
def _proc_patched(node, mod):
    ann = node.args.args[0].annotation
    if hasattr(ann, 'elts'): ann = ann.elts[0]
    cls = getattr(mod, ann.id)
    sym = getattr(cls, node.name)
    _update_func(node, sym)

In [ ]:
tree = _proc_mod(mod)
tree.body[9]

_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>


```python
@patch
def k(self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'):
    ...
```

## Class and file

In [ ]:
tree = _get_tree(mod)
node = tree.body[7]
node

```python
class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
```

In [ ]:
node.body

[@delegates(j)
 def h(self, b: bool=False, **kwargs):
     a = 1
     return a]

In [ ]:
#| export
def _proc_class(node, mod):
    cls = getattr(mod, node.name)
    _proc_tree(node, cls)

In [ ]:
tree = _proc_mod(mod)
tree.body[7]

```python
class A:

    def h(self, b: bool=False, *, d: str='a'):
        ...
```

In [ ]:
#| export
def create_pyi(fn, package=None):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    fn = Path(fn)
    mod = imp_mod(fn, package=package)
    tree = _proc_mod(mod)
    res = unparse(tree)
    fn.with_suffix('.pyi').write_text(res)

In [ ]:
create_pyi(fn)

In [ ]:
# fn = Path('/Users/jhoward/git/fastcore/fastcore/docments.py')
# create_pyi(fn, 'fastcore')

## Script

In [ ]:
#| export
from fastcore.script import call_parse

In [ ]:
#| export
@call_parse
def py2pyi(fname:str,  # The file name to convert
           package:str=None  # The parent package
          ):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    create_pyi(fname, package)

In [ ]:
#| export
@call_parse
def replace_wildcards(
    # Path to the Python file to process
    path: str):
    "Expand wildcard imports in the specified Python file."
    path = Path(path)
    path.write_text(expand_wildcards(path.read_text()))

# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()